# Get Data from Airtable

This gets the data from the two tables and formats it into dataframes. It's probably not the cleanest thing in the world, but hopefully it'll do for now.

In [77]:
import ast
import pandas as pd
import requests
from graphviz import Source
import textwrap
import simplejson

In [78]:
def mergeD(ugly_data):
    f = ugly_data["fields"]
    i = {"id": ugly_data["id"]}
    f.update(i)
    return f

def get_table(table_name):
    """ """
    auth_headers = {'Authorization': 'Bearer keyYXkjYFw61SeWDk'}
    table_url = 'https://api.airtable.com/v0/appuaXpFiadmP89sq/' + table_name
    r = requests.get(table_url, headers=auth_headers)
    d = eval(ast.literal_eval(str(r.content)[1:])) # this is almost certainly not the right way to do this!
    
    rows = [mergeD(x) for x in d["records"]]
    data_frame = pd.DataFrame(rows)
    return data_frame

In [88]:
project_data = get_table("Techs")
project_data.set_index("id", inplace=True)

node_description_pattern = "{name} [label=\"{longName}\"];"
edge_pattern = "{from_node} -> {to_node};"

node_descriptions = []
edges = []
for i, row in project_data.iterrows():
    try:
#         ln = textwrap.fill(row["LongName"], 21)
        ln = "\n".join( textwrap.wrap(row["LongName"], 15))
    except Exception as e:
        ln = row["LongName"]
    n = node_description_pattern.format(name=row["Name"], longName=ln)
    node_descriptions.append(n)
    if type(row["precursors"]) is not float: # nothing reads as NaN, therefore float
        precursors = row["precursors"]#.split(",")
        for precursor in precursors:
            e = edge_pattern.format(from_node=project_data.loc[precursor].Name,
                                    to_node=row["Name"])
            edges.append(e)

g = """digraph G {{
  rankdir  = "LR";
  label    = "Tech Tree of OATMEAL Projects";
  labelloc = "t";
  node [style=filled,
        color="black", 
        fontcolor="white", 
        shape="oval", 
        fixedsize=true, 
        width=1,
        height=1,
        fontsize=8 
       ];
  edge [arrowhead=vee,
        arrowsize=0.5]
  
  {nodes}
  
  {edges}
  {{rank = same; data; basicRobot; hardware; systemReef; SNAawareness}}
  
}}""".format(edges="\n  ".join(edges), nodes="\n  ".join(node_descriptions))


# print(g)
g_lines = g.split("\n ")

digraph_js = """var digraph_details = {lines};

  digraph_details =  digraph_details.join("\\n");
  
""".format(lines=g_lines)

with open('digraph.js', 'w') as f:
    f.write(digraph_js)

In [80]:
# make the dictionary have the project name as a key, instead of bing a list of independent dicts
simple_dict = project_data.to_dict(orient="records")
new_dict = {}
for row in simple_dict:
    new_dict[row["Name"]] = row

with open('parallelData.js', 'w') as f:
    j = simplejson.dumps(new_dict, ignore_nan=True, sort_keys=True, indent=2)
    f.write("var parallelData = {json};".format(json=j))

In [81]:
def get_thumb(row):
    try:
        h = row["Headshot"]
        if type(h) is not float:
            # TODO: make this check for small as well
            x = list(h)
            return x[0]['thumbnails']["large"]["url"]
        else:
            return "http://getdrawings.com/images/casper-drawing-8.jpg"
    except Exception as e:
        print(e)
        return "http://getdrawings.com/images/casper-drawing-8.jpg"


In [84]:
people_data = get_table("People")
people_data["thumb_large"] = people_data.apply(get_thumb, axis=1)
people_data.sample(5)

,Headshot,Initials,Name,Notes,Techs,id,thumb_large
4,NaN,RX,AI Richard,NaN,NaN,recFUgthTUYQnR7Mz,http://getdrawings.com/images/casper-drawing-8...
3,"[{'thumbnails': {'large': {'height': 500, 'url...",BaD,Barry D,NaN,NaN,recFKWrjwOUM5oWxI,https://dl.airtable.com/4NWLSnTFRwSfdFDBAt9A_l...
14,"[{'thumbnails': {'large': {'height': 500, 'url...",MB,Matthew,NaN,NaN,recwl2PnrQ8ueQBQ7,https://dl.airtable.com/yXqIqKWoQcWED51FYjOO_l...
5,"[{'thumbnails': {'large': {'height': 500, 'url...",NT,Ninotschka,NaN,[rec0tbY526abDXcRr],recHLT6IoCZXwwtp3,https://dl.airtable.com/n5K9IIPLSEiPPcY8qKCG_l...
12,"[{'thumbnails': {'large': {'height': 500, 'url...",CB,Chris B,NaN,[rectkEzpAV9fxHPwS],reck4uCQdMmA9Dy0e,https://dl.airtable.com/1ocVojsCTZJYDRTtp7kI_l...


In [83]:
# make the dictionary have the project name as a key, instead of bing a list of independent dicts
simple_people_dict = people_data.to_dict(orient="records")
new_p_dict = {}
for row in simple_people_dict:
    new_p_dict[row["id"]] = row
    
new_p_dict


with open('peopleData.js', 'w') as f:
    j = simplejson.dumps(new_p_dict, ignore_nan=True, sort_keys=True, indent=2)
    f.write("var peopleData = {json};".format(json=j))

![](https://dl.airtable.com/vao3Ge65RXuQs17gLU4p_small_MVIMG_20180224_173642.jpg)